In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# base
# large
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

C:\Users\Miki\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=4, width=200, depth=None, stream=None, compact=False, sort_dicts=False)

In [3]:
import json

with open('szinonimak.json', encoding='utf-8') as f:
    d = json.load(f)

In [4]:
pp.pprint(d)

{   'bibi': [   '(biz)',
                [   'babrál',
                    'vacakol',
                    'piszkál',
                    'pepecsel',
                    'foglalatoskodik',
                    'piszmog',
                    'babirgál',
                    'bizgelődik',
                    'csecsebecsél',
                    'kotorászik',
                    'piszmolódik',
                    'vesződik',
                    'bajlódik',
                    'erőlködik',
                    'kínlódik',
                    'sérülés',
                    'sebesülés',
                    'szúrás',
                    'sebhely',
                    'horzsolás',
                    'zúzódás',
                    'fakadékocska',
                    'bibe',
                    'akadály',
                    'baj',
                    'bökkenő',
                    'nehézség',
                    'gát',
                    'torlasz',
                    'bajoskodik',
               

In [5]:
wordsli = []
for key in d.keys():
    wordsli.append(key)
    for syn in d[key][1]:
        wordsli.append(syn)

In [7]:
import numpy as np
import json 

# memerror at 100,50
STEP=25
inputs = tokenizer(wordsli, return_tensors="pt",padding=True)
sub_inputs = {}
embsli=[]

i=0
while i+STEP<len(wordsli):
    sub_inputs["input_ids"] = inputs["input_ids"][i:i+STEP]
    sub_inputs["attention_mask"] = inputs["attention_mask"][i:i+STEP]
    i=i+STEP
    print(i)
    outputs = model(**sub_inputs)
    embsli.extend(np.mean(outputs.logits.detach().numpy(),axis=1))
if(i+STEP>len(wordsli)):
    sub_inputs["input_ids"] = inputs["input_ids"][i:]
    sub_inputs["attention_mask"] = inputs["attention_mask"][i:]
    i=len(wordsli)-1
    print(i)
    outputs = model(**sub_inputs)
    embsli.extend(np.mean(outputs.logits.detach().numpy(),axis=1))

25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
2000
2025
2050
2075
2100
2125
2150
2175
2200
2225
2250
2275
2300
2325
2350
2375
2400
2425
2450
2475
2500
2525
2550
2575
2600
2625
2650
2675
2700
2725
2750
2775
2800
2825
2850
2875
2900
2925
2950
2975
3000
3025
3050
3075
3100
3125
3150
3175
3200
3225
3250
3275
3300
3325
3350
3375
3400
3425
3450
3475
3500
3525
3550
3575
3600
3625
3650
3675
3700
3725
3750
3775
3800
3825
3850
3875
3900
3925
3950
3975
4000
4025
4050
4075
4100
4125
4150
4175
4200
4225
4250
4275
4300
4325
4350
4375
4400
4425
4450
4475
4500
4525
4550
4575
4600
4625
4650
4675
4700
4725
4750
4775
4800
4825
4850
4875
4900
4925
4950
4975
5000
5025
5050
5075
5100
5125
5150
5175
5200
52

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 750006000 bytes.

In [ ]:
parts_tupli=[]
cnt=0
for w in wordsli:
    parts_tupli.append((w,embsli[cnt]))
    cnt=cnt+1
    
i=0
big_d={}
for k,v in d.items():

        new_val_li=[]
        key_emb=parts_tupli[i][1]
        i=i+1
        for e in v[1]:

                new_val_li.append(parts_tupli[i])
                i=i+1
        big_d[k]=(key_emb,new_val_li)

In [ ]:
# res print
for k,v in big_d.items():
    print(k)
    for e in v[1]:
        print('\t',e[0])

In [ ]:
# measurement
from scipy.spatial import distance

sum=0
cnt=0
avg=0

for k,v in big_d.items():
    emb_baseword=v[0]
    for e in v[1]:
        emb_synonym=e[1]

        dist=distance.cosine(emb_baseword,emb_synonym)
        sum=sum+dist
        cnt=cnt+1
# overall result on synonym-dictionary       
avg=sum/cnt
print(avg)
# buffered: xlm roberta large(m) 0. --- xlm roberta base (m) 0.
# one-by-one: xlm roberta large (170m) 0.0025427154258488224   ---   xlm roberta base (63m) 0.002523067917173937

# measurement,automated, old
from scipy.spatial import distance

sum=0
cnt=0
avg=0

for k,v in d.items():
    baseword=k
    encoded_input_base=tokenizer(baseword,return_tensors='pt')
    output_base = model(**encoded_input_base)
    for e in v[1]:
        synonym=e
        encoded_input_synonym=tokenizer(synonym,return_tensors='pt')
        output_synonym = model(**encoded_input_synonym)

        embedded_base=output_base[0][0][0]
        embedded_synonym=output_synonym[0][0][0]

        dist=distance.cosine(embedded_base.detach().numpy(),embedded_synonym.detach().numpy())
        sum=sum+dist
        cnt=cnt+1
# overall result on synonym-dictionary       
avg=sum/cnt
print(avg)
# xlm roberta large (170m) 0.0025427154258488224   ---   xlm roberta base (63m) 0.002523067917173937

# old way, just two words, manually
hu1 = "fiú"
encoded_input1 = tokenizer(hu1, return_tensors='pt')
output1 = model(**encoded_input1)
emb_hu1=output1[0][0][0]

hu2 = "férfi"
encoded_input2 = tokenizer(hu2, return_tensors='pt')
output2 = model(**encoded_input2)
emb_hu2=output2[0][0][0]

from scipy.spatial import distance

print(distance.cosine(emb_hu1.detach().numpy(),emb_hu2.detach().numpy())) #large 0.0007125139236450195 fiu,ferfi  -  base 0.002390563488006592 fiu,ferfi